In [1]:
from selenium import webdriver
import shutil
import os
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Set the download directory path
script_name = 'attachments_samiksha_shinde1'
folder_name = '/' + script_name
download_directory = 'C:/Users/samiksha-shinde/my_new_folder'

tmp_dwn = download_directory + folder_name
tmp_dwn_dir = tmp_dwn.replace('/',"\\")
os.makedirs(tmp_dwn_dir)
# Create a dictionary to set download preferences
profile = {
    "download.default_directory": tmp_dwn_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("prefs", profile)

driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
# driver1 = webdriver.Chrome(chrome_options=chrome_options)
# Navigate to a page where the file can be downloaded
driver.get("https://www.tenderned.nl/aankondigingen/overzicht/327158")
time.sleep(5)

# notice_url = driver.find_element(By.CSS_SELECTOR, 'a.tn-link').get_attribute("href")
# driver1.get(notice_url)
# time.sleep(5)

Publicatie_click = WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.XPATH,'//*[contains(text(),"Publicatie")]')))
driver.execute_script("arguments[0].click();",Publicatie_click)
time.sleep(7) 


external_url = driver.find_element(By.XPATH,'//*[@id="mat-tab-content-0-1"]/div/tn-aankondiging-publicatie-tab/tn-secondary-button/div/button').click()
time.sleep(3)
# Perform actions to trigger the file download (replace with your own actions)

# Wait for some time to ensure the download completes
# time.sleep(5)

# # Get the list of files in the download directory
files = os.listdir(download_directory)
print(files)

# Close the WebDriver
# driver.quit()
# driver1.quit()


['attachments', 'attachments_sami', 'attachments_samiksha_shinde', 'attachments_samiksha_shinde1', 'my_file.py', 'my_new_file']


In [2]:
from mydir1.master_class import *
from mydir1 import master_functaion as fn
import mydir1.json_file
import logging
import time
import shutil
import os
import re
import jsons
from datetime import date, datetime, timedelta
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By
SCRIPT_NAME = "in_riebbs_spn"
# output_json_folder = "jsonfile"
output_json_file = mydir1.json_file.OutputJSON(SCRIPT_NAME)

#***************************************************************
# Set the download directory path
# script_name = 'attachments_sami'
# folder_name = '/' + script_name
# download_directory = 'C:/Users/samiksha-shinde/my_new_folder'

# tmp_dwn = download_directory + folder_name
# tmp_dwn_dir = tmp_dwn.replace('/',"\\")
# os.makedirs(tmp_dwn_dir)
# # Create a dictionary to set download preferences
# profile = {
#     "download.default_directory": tmp_dwn_dir,
#     "download.prompt_for_download": False,
#     "download.directory_upgrade": True,
#     "safebrowsing.enabled": True
# }

#*****************************************************************
def extract_and_save_notice(tender_html_element):
    notice_data = master_data()
    
    
    try:
        notice_data.local_title = tender_html_element.find_element(By.CSS_SELECTOR, 'span.tn-h3 > a').text
        notice_data.notice_title = GoogleTranslator(source='auto', target='en').translate(notice_data.local_title)
        print('notice_data.local_title::',notice_data.local_title)
    except Exception as e:
        logging.info("Exception in local_title: {}".format(type(e).__name__))
        pass
        
    # Onsite Comment -split and take only date and take time also if available  29 jan. 2024 
    try:
        publish_date = tender_html_element.find_element(By.CSS_SELECTOR, 'mat-card-header > div > mat-card-subtitle > div:nth-child(2)').text.split('-')[0].strip()
        publish_date = re.findall('\d+ \w+. \d{4}',publish_date)[0]
        notice_data.publish_date = datetime.strptime(publish_date,'%d %b. %Y').strftime('%Y/%m/%d %H:%M:%S')
        logging.info(notice_data.publish_date)
    except Exception as e:
        logging.info("Exception in publish_date: {}".format(type(e).__name__))
        pass
    
    
    

    
    notice_data.master_cleanup()
    json_object = jsons.dump(notice_data)
#     print(json_object)
    output_json_file.writeNoticeToJSONFile(json_object)
    
    
#*****************************************************************
arguments= ['--incognito','ignore-certificate-errors','allow-insecure-localhost','--start-maximized','--no-sandbox','--disable-dev-shm-usage']
browser1= fn.drivers(arguments)
browser2= fn.drivers(arguments)
urls = ["https://www.tenderned.nl/aankondigingen/overzicht"] 

for url in urls:
    fn.get_url(browser1,url,50)
    print(url)
    
    next_page = WebDriverWait(page_main, 50).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="mat-select-0"]/div/div[2]')))
    page_main.execute_script("arguments[0].click();",next_page)

    next_page = WebDriverWait(page_main, 50).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="mat-option-1"]/mat-pseudo-checkbox')))
    page_main.execute_script("arguments[0].click();",next_page)
    time.sleep(10)
    
    count_records = 0
    for page_no in range(2,5):
        page_check = WebDriverWait(browser1, 50).until(EC.presence_of_element_located((By.XPATH,'//*[@id="app"]/tn-aankondigingen-page/div[2]/div/tn-aankondiging-overzicht/mat-drawer-container/mat-drawer-content/div[2]/div[2]/mat-card'))).text
        rows = WebDriverWait(browser1, 60).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app"]/tn-aankondigingen-page/div[2]/div/tn-aankondiging-overzicht/mat-drawer-container/mat-drawer-content/div[2]/div[2]/mat-card')))
        length = len(rows)
        for records in range(1,5):
            tender_html_element = WebDriverWait(browser1, 60).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app"]/tn-aankondigingen-page/div[2]/div/tn-aankondiging-overzicht/mat-drawer-container/mat-drawer-content/div[2]/div[2]/mat-card')))[records]
            extract_and_save_notice(tender_html_element)
            count_records += 1
            
#         try:   
#             next_page = WebDriverWait(browser1, 50).until(EC.element_to_be_clickable((By.LINK_TEXT,str(page_no))))
#             browser1.execute_script("arguments[0].click();",next_page)
#             logging.info("Next page")
#             WebDriverWait(browser1, 50).until_not(EC.text_to_be_present_in_element((By.CSS_SELECTOR,'body > section.about_bg > div:nth-child(1) > div > div > table > tbody > tr:nth-child(3)'),page_check))
#         except Exception as e:
#             logging.info("Exception in next_page: {}".format(type(e).__name__))
#             logging.info("No next page")
#             break
            
    print(count_records)
    output_json_file.copyFinalJSONToServer()
# browser1.quit()
print('********************************************************************************************************')

https://www.tenderned.nl/aankondigingen/overzicht


NameError: name 'page_main' is not defined